In [9]:
#!pip install transformers
#!pip install huggingface_hub
#!pip install datasets
#!pip install torch
!pip install transformers[torch]

     -------------------------------------- 227.6/227.6 kB 4.6 MB/s eta 0:00:00


In [17]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
datasets = load_dataset("piqa")

Found cached dataset piqa (C:/Users/shwet/.cache/huggingface/datasets/piqa/plain_text/1.1.0/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
datasets.keys()

dict_keys(['train', 'test', 'validation'])

In [21]:
datasets["train"][0]["goal"]

"When boiling butter, when it's ready, you can"

In [22]:
def preprocess_intents_json(segment):
    
    preprocessed_data = []
    
    for data in datasets[segment]:
        preprocessed_data.append(f"Goal: {data['goal']}\n")
        if data['label']=='1':
            preprocessed_data.append(f"Solution: {data['sol2']}\n")
        else:
            preprocessed_data.append(f"Solution: {data['sol1']}\n")
    
    return "".join(preprocessed_data)

def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)


output_file = f"./data/data_train.txt"
preprocessed_data = preprocess_intents_json("train")
preprocessed_data = re.sub(r'\n+', '\n', preprocessed_data).strip() 
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_validation.txt"
preprocessed_data = preprocess_intents_json("validation")
preprocessed_data = re.sub(r'\n+', '\n', preprocessed_data).strip() 
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_test.txt"
preprocessed_data = preprocess_intents_json("test")
preprocessed_data = re.sub(r'\n+', '\n', preprocessed_data).strip() 
save_preprocessed_data(preprocessed_data, output_file)

## Preprocessing Data

In [23]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [24]:
train_file_path = "./data/data_train.txt"
test_file_path = "./data/data_test.txt"
model_name = "gpt2"
output_dir = "gpt2-fine-tuned"
batch_size = 2
num_train_epochs = 5

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_file_path, block_size=512) 
test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_file_path, block_size=512)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps=500,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir='./logs',
    gradient_accumulation_steps=1,
    learning_rate=1e-4,
)

c:\Users\shwet\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shwet\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\shwet\anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the l

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [26]:
trainer.train()

  0%|          | 0/2950 [00:00<?, ?it/s]